# CMS Jet Data Machine Learning with Arc Kernel

### Import packages

In [8]:
import os
import jet
import uproot
import awkward as ak

import pennylane as qml
import pennylane.numpy as np

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme()

GPU = 'cuda' if torch.cuda.is_available() else 'cpu'

### Model structure

##### Classical layers and models

In [9]:
class ClassicalModel(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, hidden_layers):
        super().__init__()
        if hidden_layers == 0:
            net = [nn.Linear(input_dim, 1)]
        else:
            net = [nn.Linear(input_dim, hidden_dim), nn.ReLU()]
            for _ in range(hidden_layers-1):
                net += [nn.Linear(hidden_dim, hidden_dim)]
                net += [nn.ReLU()]
            net += [nn.Linear(hidden_dim, 1)]
        # BCEWithLogitsLoss already contains a sigmoid function
        self.net = nn.Sequential(*net)
    def forward(self, x):
        y = self.net(x)
        return y

##### Quantum layers and models

In [10]:
def encode_daughter_pt_ratio_delta(inputs, cluster_r=1):
    inputs = inputs.reshape((3, -1))
    num_particles = inputs.shape[1]
    num_qubits = 3 * num_particles
    norm_pt, norm_eta, norm_phi = inputs
    for ptc in range(num_particles):
        qml.RY(2 * torch.asin(norm_pt[ptc]), wires=3*ptc)
        qml.RY(2 * torch.asin(norm_pt[ptc]), wires=3*ptc+1)
        qml.CRY(2 * torch.asin(norm_eta[ptc]), wires=[3*ptc, 3*ptc+2])
        qml.CRY(2 * torch.asin(norm_phi[ptc]), wires=[3*ptc+1, 3*ptc+2])

def qml_torch_layer(num_qubits, weight_shapes, enc_layer, qml_layer):
    dev = qml.device('default.qubit', wires=num_qubits)
    @qml.qnode(dev)
    def qnode(inputs, weights):
        enc_layer(inputs)
        qml_layer(weights=weights, wires=range(num_qubits))
        return [qml.expval(qml.PauliZ(wires=i)) for i in range(num_qubits)]
    return qml.qnn.TorchLayer(qnode, weight_shapes)

class HybridModel(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, hidden_layers, num_qubits, weight_shapes, enc_layer, qml_layer):
        super().__init__()
        self.num_particles = num_qubits // 3
        self.qkernel = qml_torch_layer(num_qubits, weight_shapes, enc_layer, qml_layer)
        self.net = ClassicalModel(input_dim+num_qubits, hidden_dim, hidden_layers)
    def forward(self, x):
        parent_pt_eta_phi = x[:, :3].reshape(-1, 3, 1)
        parent_pt, parent_eta, parent_phi = parent_pt_eta_phi[:, 0], parent_pt_eta_phi[:, 1], parent_pt_eta_phi[:, 2]
        daughter_pt_eta_phi = x[:, -self.num_particles*3:].reshape(-1, 3, self.num_particles)
        daughter_pt, daughter_eta, daughter_phi = daughter_pt_eta_phi[:, 0], daughter_pt_eta_phi[:, 1], daughter_pt_eta_phi[:, 2]
        pt_ratio, delta_eta, delta_phi = daughter_pt/parent_pt, daughter_eta-parent_eta, daughter_phi-parent_phi
        delta_r, cluster_radius = torch.sqrt(delta_eta**2 + delta_phi**2), 1
        norm_pt  = pt_ratio * delta_r / cluster_radius
        norm_eta = delta_eta / delta_r
        norm_phi = delta_phi / delta_r
        assert (torch.abs(norm_pt) <= 1).all() and (torch.abs(norm_eta) <= 1).all() and  (torch.abs(norm_phi) <= 1).all(), \
            f"Circuit Error : Recieve argument greater than 1 in torch.asin ({torch.abs(norm_pt)}, {torch.abs(norm_eta)}, {torch.abs(norm_phi)})"
        circuit_input = torch.cat((norm_pt, norm_eta, norm_phi), dim=1)
        x = torch.cat((x, self.qkernel(circuit_input)), dim=1)
        y = self.net(x)
        return y

### Training procedure

In [11]:
def train(model, data_loader, config):
    model = model.to(GPU)
    loss = nn.BCEWithLogitsLoss(reduction="mean")
    opt = torch.optim.Adam(model.parameters(), lr=config["learning_rate"], weight_decay=config["weight_decay"])
    record = {"train_loss":[], "train_acc":[], "test_loss":[], "test_acc":[]}
    for epoch in range(config["num_epochs"]):
        train_loss, train_acc, test_loss, test_acc = 0, 0, 0, 0
        model.train()
        for x, y_true in data_loader["train"]:
            x, y_true = x.to(GPU), y_true.to(GPU)
            opt.zero_grad()
            y_pred = model(x)
            batch_loss = loss(y_pred, y_true)
            batch_loss.backward()
            train_loss += batch_loss
            # BCEWithLogitsLoss -> >=0 | Sigmoid + BCELoss -> >=0.5
            train_acc += torch.sum((y_pred >= 0) == y_true) / len(x)
            opt.step()
        model.eval()
        for x, y_true in data_loader["test"]:
            x, y_true = x.to(GPU), y_true.to(GPU)
            y_pred = model(x)
            batch_loss = loss(y_pred, y_true)
            test_loss += batch_loss * len(x)
            # BCEWithLogitsLoss -> >=0 | Sigmoid + BCELoss -> >=0.5
            test_acc += torch.sum((y_pred >= 0) == y_true).item()
        train_loss /= len(data_loader["train"])
        train_acc /= len(data_loader["train"])
        test_loss /= len(data_loader["test"].dataset)
        test_acc /= len(data_loader["test"].dataset)
        record["train_loss"].append(train_loss)
        record["train_acc"].append(train_acc)
        record["test_loss"].append(test_loss)
        record["test_acc"].append(test_acc)
        print(f"Epoch {epoch+1} : train = (loss:{train_loss:.2f}, acc:{train_acc:.2f}) | test = (loss:{test_loss:.2f}, acc:{test_acc:.2f})")
    return record

### Data

In [12]:
def load_data_buffer(channel, get_method, *args):
    suffix = " ".join(map(str, args))
    buffer_file = f"data_buffer/{channel}-{get_method.__name__}-{suffix}.pt"
    if not os.path.exists(buffer_file):
        print(f"Buffer ({get_method.__name__}) : {channel}.pt not found, create now ...")
        events = get_method(channel, *args)
        torch.save(events, buffer_file)
    else:
        events = torch.load(buffer_file)
        print(f"Buffer ({get_method.__name__}) : {channel}.pt found, loading complete!")
    return events

def get_events(channel, num_events, num_particles, jet_type, cut):
    jet_parent = load_data_buffer(channel, jet.get_parent_info, num_events, jet_type, cut)
    if num_particles >= 1:
        jet_daughter = load_data_buffer(channel, jet.get_daughter_info, num_events, num_particles, jet_type, cut)
        return torch.cat((jet_parent, jet_daughter), dim=1)
    else:
        return jet_parent

jet_type   = "fatjet"
num_events = 1000
num_particles = 3
cut = f"({jet_type}_pt >= 500) & ({jet_type}_pt <= 1500)"

signal_channel = "ZprimeToZhToZinvhbb"
# signal_channel = "ZprimeToZhToZlephbb"
# background_channel = "QCD_HT1500to2000"
background_channel = "QCD_HT2000toInf"

data_ratio = 0.9
signal_events = get_events(signal_channel, num_events, num_particles, jet_type, cut)
background_events = get_events(background_channel, num_events, num_particles, jet_type, cut)
num_sig, num_bkg = len(signal_events), len(background_events)
num_data = min(num_sig, num_bkg)
num_train = int(data_ratio * num_data)
num_test = num_data - num_train
print("-" * 100)
print(f"Signal = {signal_channel} | Background = {background_channel} | Cut = {cut}")
print(f"Length Signal = {num_sig} | Length Background = {num_bkg} | Number of Data = {num_data} | Shape = {signal_events.shape}")

Buffer (get_parent_info) : ZprimeToZhToZinvhbb.pt not found, create now ...
JetEvents : Successfully open /Users/yianchen/CMS_Open_Data_Workspace/CMSSW_7_6_7/src/QCD_Jet_Fatjet/Analyzer/root_files/ZprimeToZhToZinvhbb_1000.root
JetEvents : Successfully create ZprimeToZhToZinvhbb with 1000 events.
Buffer (get_daughter_info) : ZprimeToZhToZinvhbb.pt not found, create now ...
JetEvents : Successfully open /Users/yianchen/CMS_Open_Data_Workspace/CMSSW_7_6_7/src/QCD_Jet_Fatjet/Analyzer/root_files/ZprimeToZhToZinvhbb_1000.root
JetEvents : Successfully create ZprimeToZhToZinvhbb with 1000 events.


get_daughter_info : Channel ZprimeToZhToZinvhbb with 1000 events: 100%|██████████| 582/582 [00:02<00:00, 278.62it/s]


Buffer (get_parent_info) : QCD_HT2000toInf.pt not found, create now ...
JetEvents : Successfully open /Users/yianchen/CMS_Open_Data_Workspace/CMSSW_7_6_7/src/QCD_Jet_Fatjet/Analyzer/root_files/QCD_HT2000toInf_1000.root
JetEvents : Successfully create QCD_HT2000toInf with 1000 events.
Buffer (get_daughter_info) : QCD_HT2000toInf.pt not found, create now ...
JetEvents : Successfully open /Users/yianchen/CMS_Open_Data_Workspace/CMSSW_7_6_7/src/QCD_Jet_Fatjet/Analyzer/root_files/QCD_HT2000toInf_1000.root
JetEvents : Successfully create QCD_HT2000toInf with 1000 events.


get_daughter_info : Channel QCD_HT2000toInf with 1000 events: 100%|██████████| 985/985 [00:03<00:00, 268.42it/s]

----------------------------------------------------------------------------------------------------
Signal = ZprimeToZhToZinvhbb | Background = QCD_HT2000toInf | Cut = (fatjet_pt >= 500) & (fatjet_pt <= 1500)
Length Signal = 582 | Length Background = 985 | Number of Data = 582 | Shape = torch.Size([582, 15])


In [13]:
class JetDataset(Dataset):
    def __init__(self, signal_events, background_events):
        self.x = torch.cat((signal_events ,background_events))
        self.y = torch.cat((torch.ones((len(signal_events)), 1), torch.zeros((len(background_events)), 1)))
        self.x.requires_grad = False
        self.y.requires_grad = False
    def __getitem__(self, index):
        return self.x[index], self.y[index]
    def __len__(self):
        return len(self.y)

data_train = JetDataset(signal_events[:num_train], background_events[:num_train])
data_test = JetDataset(signal_events[num_train:], background_events[num_train:])

### Result

In [14]:
cf = {
    "learning_rate":1E-3,
    "weight_decay":0,
    "num_epochs":30,
    "batch_size":64,
}

data_loader = {
    "train":DataLoader(data_train, cf["batch_size"], shuffle=True, drop_last=True),
    "test":DataLoader(data_test, cf["batch_size"], shuffle=False, drop_last=False),
    }

input_dim = signal_events.shape[1]
hidden_dim, hidden_layers = 10 * input_dim, 2
num_qubits = 3 * num_particles
weight_shapes = {"weights" : (2, num_qubits)}
enc_layer = encode_daughter_pt_ratio_delta
qml_layer = qml.BasicEntanglerLayers

c_model = ClassicalModel(input_dim, hidden_dim, hidden_layers)
# q_model = HybridModel(input_dim, hidden_dim, hidden_layers, num_qubits, weight_shapes, enc_layer, qml_layer)

c_result = train(c_model, data_loader, cf)
# q_result = train(q_model, data_loader, cf)

# np.save(f"result_arckernel/c_{jet_type}-{signal_channel} vs {background_channel}-{num_particles}-{num_events}-{cut}.npy", c_result)
# np.save(f"result_arckernel/q_{jet_type}-{signal_channel} vs {background_channel}-{num_particles}-{num_events}-{cut}.npy", q_result)

Epoch 1 : train = (loss:4.84, acc:0.51) | test = (loss:0.59, acc:0.89)
Epoch 2 : train = (loss:1.44, acc:0.55) | test = (loss:0.98, acc:0.42)
Epoch 3 : train = (loss:0.83, acc:0.55) | test = (loss:0.67, acc:0.56)
Epoch 4 : train = (loss:0.70, acc:0.58) | test = (loss:0.80, acc:0.36)
Epoch 5 : train = (loss:0.68, acc:0.58) | test = (loss:1.18, acc:0.16)
Epoch 6 : train = (loss:0.82, acc:0.55) | test = (loss:1.52, acc:0.17)
Epoch 7 : train = (loss:0.82, acc:0.54) | test = (loss:0.42, acc:0.86)
Epoch 8 : train = (loss:0.87, acc:0.57) | test = (loss:1.02, acc:0.28)
Epoch 9 : train = (loss:0.75, acc:0.55) | test = (loss:0.86, acc:0.41)
Epoch 10 : train = (loss:0.79, acc:0.55) | test = (loss:1.39, acc:0.17)
Epoch 11 : train = (loss:0.78, acc:0.56) | test = (loss:1.13, acc:0.26)
Epoch 12 : train = (loss:0.72, acc:0.58) | test = (loss:0.47, acc:0.84)
Epoch 13 : train = (loss:0.72, acc:0.57) | test = (loss:0.56, acc:0.70)
Epoch 14 : train = (loss:0.76, acc:0.58) | test = (loss:0.55, acc:0.69)
E